In [1]:
pip install torch numpy scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [3]:

# Define the neural network
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        # Input layer to hidden layer
        self.hidden = nn.Linear(2, 5)  # Assuming input features are of size 2, and hidden layer has 5 nodes
        self.hidden2 = nn.Linear(5, 8)
        # Hidden layer to output layer
        self.output = nn.Linear(8, 1)  # Output layer for binary classification (1 node)

    def forward(self, x):
        # Forward pass: compute the output of the network
        a1 = F.relu(self.hidden(x))
        a2 = F.relu(self.hidden2(a1))
        x = torch.sigmoid(self.output(a2))  # Activation function for output layer for binary classification
        return x

In [4]:
# Create an instance of the network
my_net = SimpleNeuralNet()
print(my_net)

# Dummy input for testing the network
dummy_input = torch.randn(1, 2)  # Batch size of 1, input features of size 2
print(dummy_input)
output = my_net(dummy_input)
print("Output of the network:", output)

SimpleNeuralNet(
  (hidden): Linear(in_features=2, out_features=5, bias=True)
  (hidden2): Linear(in_features=5, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=1, bias=True)
)
tensor([[ 0.1498, -0.2089]])
Output of the network: tensor([[0.5534]], grad_fn=<SigmoidBackward0>)


In [5]:
# Synthetic non-linear dataset
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=600, noise=0.5, random_state=42)  # Create non-linear data
inputs = torch.tensor(X, dtype=torch.float)  # Convert to tensors
labels = torch.tensor(y, dtype=torch.float).view(-1, 1)

# Splitting dataset into train and test
train_inputs, test_inputs = inputs[:540], inputs[540:]
train_labels, test_labels = labels[:540], labels[540:]

criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = torch.optim.Adam(my_net.parameters(), lr=0.01)  # Stochastic Gradient Descent

# Training loop (for demonstration, let's say 100 epochs)
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    predicted_labels = my_net(inputs)

    # Compute and print loss
    loss = criterion(predicted_labels, labels)
    if epoch % 10 == 0:  # Print every 10 epochs
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Print final loss
print(f'Final Loss: {loss.item()}')

Epoch 0, Loss: 0.6921742558479309
Epoch 10, Loss: 0.5985599160194397
Epoch 20, Loss: 0.4870801866054535
Epoch 30, Loss: 0.4468354880809784
Epoch 40, Loss: 0.4401010274887085
Epoch 50, Loss: 0.4399012327194214
Epoch 60, Loss: 0.43844524025917053
Epoch 70, Loss: 0.4376726746559143
Epoch 80, Loss: 0.43708038330078125
Epoch 90, Loss: 0.4363258481025696
Final Loss: 0.43558114767074585


# Task 1: 
a. Implement the predict function by running the prediction through neural network and return the predicted labels.
b. Compute the accuracy for the test set.

In [6]:
def predict(x):
    # Apply the network to the input
    predicted = my_net(x)
    # Convert the output to binary
    predicted = (predicted > 0.5).float()
    return predicted

In [7]:
def accuracy(y_pred, y_true):
    # Compare the predicted labels to true labels
    return (y_pred == y_true).float().mean()

In [8]:
test_pred = predict(test_inputs)
test_accuracy = accuracy(test_pred, test_labels)

In [9]:
test_accuracy

tensor(0.7500)

# Task 2
a. Change the network to use ReLU activation function, and train again
b. Change the optimization algorithm to Adam and train again
c. Add an additional hidden layer and train again

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the neural network
class SimpleNeuralNet(nn.Module):
    def __init__(self, num_hidden_layers, activation_function, input_size=2, hidden_sizes=[5, 8], output_size=1):
        super(SimpleNeuralNet, self).__init__()
        
        self.activation_function = activation_function
        self.layers = nn.ModuleList()

        # Add input layer
        if num_hidden_layers > 0:
            self.layers.append(nn.Linear(input_size, hidden_sizes[0]))

        # Add hidden layers
        for i in range(1, num_hidden_layers):
            self.layers.append(nn.Linear(hidden_sizes[i - 1], hidden_sizes[i]))

        # Add output layer
        self.layers.append(nn.Linear(hidden_sizes[-1], output_size))

    def forward(self, x):
        # Forward pass: compute the output of the network
        for i in range(len(self.layers) - 1):  # Loop over all layers except the output layer
            x = self.layers[i](x)
            if self.activation_function == 'relu':
                x = F.relu(x)
            elif self.activation_function == 'sigmoid':
                x = torch.sigmoid(x)
            elif self.activation_function == 'tanh':
                x = torch.tanh(x)
            # Add more activation functions as needed

        # Activation function for output layer for binary classification
        x = torch.sigmoid(self.layers[-1](x)) 
        return x

    def predict(self, x):
        # Apply the network to the input
        predicted = self(x)
        # Convert the output to binary
        predicted = (predicted > 0.5).float()
        return predicted

# Example usage



In [11]:
nn_model = SimpleNeuralNet(num_hidden_layers=2, activation_function='relu')

In [12]:
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass: Compute predicted y by passing x to the model
    predicted_labels = nn_model(inputs)

    # Compute and print loss
    loss = criterion(predicted_labels, labels)
    if epoch % 10 == 0:  # Print every 10 epochs
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Print final loss
print(f'Final Loss: {loss.item()}')

Epoch 0, Loss: 0.6806690692901611
Epoch 10, Loss: 0.6806690692901611
Epoch 20, Loss: 0.6806690692901611
Epoch 30, Loss: 0.6806690692901611
Epoch 40, Loss: 0.6806690692901611
Epoch 50, Loss: 0.6806690692901611
Epoch 60, Loss: 0.6806690692901611
Epoch 70, Loss: 0.6806690692901611
Epoch 80, Loss: 0.6806690692901611
Epoch 90, Loss: 0.6806690692901611
Final Loss: 0.6806690692901611


In [13]:
test_pred = nn_model.predict(test_inputs)
test_accuracy = accuracy(test_pred, test_labels)

In [14]:
test_accuracy

tensor(0.5333)